In [ ]:
!pip3 install transformers
!pip3 install torch torchvision
!pip3 install datasets
!pip3 install tqdm
!pip3 install ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1

In [ ]:
import transformers
import torch
import datasets
from datasets import concatenate_datasets, load_dataset
from tqdm.auto import tqdm
from transformers import BertTokenizer, DataCollatorWithPadding, AdamW,\
      BertForMaskedLM, get_scheduler
import random

In [ ]:
# Data colllection and pre-processing for Next Sentence Prediction
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

law_list = ['acus_reports', 'cc_casebooks', 'cfpb_creditcard_contracts',
            'ed_policy_guidance', 'eoir', 
            'frcp', 'fre', 'ftc_advisory_opinions', 'hhs_alj_opinions', 
            'uscode']
law_list1 = ['acus_reports', 'cc_casebooks', 'cfpb_creditcard_contracts',
            'ed_policy_guidance', 'eoir', 
            'frcp', 'fre', 'ftc_advisory_opinions', 'hhs_alj_opinions', 
            'uscode', "cfr", "scotus_oral_arguments", "state_codes"] 

dataset = [load_dataset('pile-of-law/pile-of-law', law, split= 'train') for law in law_list1]

n = len(dataset)
def get_mini_dataset(dataset, subset_size=1):
    subset = [None] * n
    # validation_subset = [] 
    t = []
#     start = time.time()
    for i in range(n):        
        # Append a new subset dataset for the `train` set
        subset[i] = dataset[i].select(range(dataset[i].num_rows//subset_size))
#         t.append(time.time()-start)
#         start = time.time()

    # Create a new DatasetDict with the subset datasets
    a = [i.num_rows//subset_size for i in dataset]
    print(a)
#     print(t)
    merged_dataset = concatenate_datasets(subset)
    subset_dataset = datasets.DatasetDict({'train': merged_dataset})#, 'validation': validation_subset})
    return subset_dataset

mini_raw_datasets = get_mini_dataset(dataset)
print(len(mini_raw_datasets['train']))


text = mini_raw_datasets['train'][:]['text']

bag = [item for sentence in text for item in sentence.replace('\n',' ').split('.') if item != '']
bag_size = len(bag)
print(bag_size)


sentence_a = []
sentence_b = []
label = []

for paragraph in text:
    sentences = [
        sentence for sentence in paragraph.replace('\n','').split('.') if sentence != ''
    ]
    num_sentences = len(sentences)
    length = 25
#     print(num_sentences)
    if num_sentences > 1:
        # while 1:
            start = random.randint(0, num_sentences-2)
            # 50/50 whether is IsNextSentence or NotNextSentence
            if random.random() >= 0.5:
                # this is IsNextSentence
                if len(sentences[start])>length or len(sentences[start+1])>length:
                    sentence_a.append(sentences[start])
                    sentence_b.append(sentences[start+1])
                    label.append(0)
                    # break
            else:
                index = random.randint(0, bag_size-1)
                if len(sentences[start])>length or len(bag[index])>length:
                    # this is NotNextSentence
                    sentence_a.append(sentences[start])
                    sentence_b.append(bag[index])
                    label.append(1)
                    # break

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
batch = tokenizer(sentence_a, sentence_b, max_length=256, padding='max_length', truncation=True, return_tensors='pt')
batch['next_sentence_label'] = torch.LongTensor([label]).T
# batch = tokenizer(sentence_a, sentence_b, truncation=False)


INFO:datasets.info:Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/pile-of-law--pile-of-law/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
INFO:datasets.builder:Overwrite dataset info from restored data version if exists.
INFO:datasets.info:Loading Dataset info from /root/.cache/huggingface/datasets/pile-of-law___pile-of-law/acus_reports/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
INFO:datasets.info:Loading Dataset info from /root/.cache/huggingface/datasets/pile-of-law___pile-of-law/acus_reports/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
INFO:datasets.info:Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/pile-of-law--pile-of-law/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
INFO:datasets.builder:Overwrite dataset info from restored data version if exists.
INFO:datasets.info:Loading Dataset info from /root/.cache/huggingf

INFO:datasets.builder:Dataset not on Hf google storage. Downloading and preparing it from source


INFO:datasets.utils.file_utils:https://huggingface.co/datasets/pile-of-law/pile-of-law/resolve/main/data/train.congressional_hearings.jsonl.xz not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/downloads/tmpevyo8pq8


INFO:datasets.utils.file_utils:storing https://huggingface.co/datasets/pile-of-law/pile-of-law/resolve/main/data/train.congressional_hearings.jsonl.xz in cache at /root/.cache/huggingface/datasets/downloads/5e1c5da6b96a57dcc27405a7baab074d0cdb697d5365a2584798f88f01edd100
INFO:datasets.utils.file_utils:creating metadata file for /root/.cache/huggingface/datasets/downloads/5e1c5da6b96a57dcc27405a7baab074d0cdb697d5365a2584798f88f01edd100
INFO:datasets.download.download_manager:Downloading took 0.0 min
INFO:datasets.download.download_manager:Checksum Computation took 0.0 min


INFO:datasets.utils.file_utils:https://huggingface.co/datasets/pile-of-law/pile-of-law/resolve/main/data/validation.congressional_hearings.jsonl.xz not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/downloads/tmpz2wws47z


...

Generating train split: 0 examples [00:00, ? examples/s]

INFO:datasets.builder:Generating validation split


Generating validation split: 0 examples [00:00, ? examples/s]

INFO:datasets.utils.info_utils:Unable to verify splits sizes.


Dataset pile-of-law downloaded and prepared to /root/.cache/huggingface/datasets/pile-of-law___pile-of-law/courtlistener_opinions/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60. Subsequent calls will reuse this data.


In [ ]:
# Pre-processing for masked LM
labels = batch['input_ids']
mask =  batch['attention_mask']
token_type_ids = batch['token_type_ids']
next_sentence_labels = batch['next_sentence_label']
# make copy of labels tensor, this will be input_ids
input_ids = labels.detach().clone()
# create random array of floats with equal dims to input_ids
rand = torch.rand(input_ids.shape)
# mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 101) * (input_ids != 102)
# loop through each row in input_ids tensor (cannot do in parallel)
for i in range(input_ids.shape[0]):
    # get indices of mask positions from mask array
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    # mask input_ids
    input_ids[i, selection] = 103  # our custom [MASK] token == 3
  


In [ ]:
# Construct dataset
encodings = {'input_ids': input_ids, 'attention_mask': mask, 'labels': labels, 'token_type_ids':token_type_ids, "next_sentence_labels": next_sentence_labels}
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

dataset = Dataset(encodings)

train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
# Training Loop
from transformers.models.bert.modeling_bert import BertForPreTraining
model = BertForPreTraining.from_pretrained('bert-base-uncased')
optim = AdamW(model.parameters(), lr=5e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# activate training mode
model.train()
# initialize optimizer

epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(train_dataloader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        next_sentence_labels = batch['next_sentence_labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels, token_type_ids=token_type_ids, next_sentence_label = next_sentence_labels)
        # extract loss
        loss, prediction_scores = outputs[:2]
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

In [ ]:
# Test simple case
test_text = "The [MASK] is an essential step in the legislative process, allowing \
representatives to propose and debate potential laws before they are enacted."
input_ids = tokenizer(test_text, return_tensors="pt")["input_ids"]
output = model(input_ids)
output
prediction_scores = output.prediction_logits
mask_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item()
mask_scores = prediction_scores[0, mask_index]
sorted_scores, sorted_indices = torch.sort(mask_scores, descending=True)
top_predictions = []
for i in range(len(sorted_scores)):
    token_id = sorted_indices[i].item()
    token = tokenizer.convert_ids_to_tokens(token_id)
    top_predictions.append((token, sorted_scores[i].item()))

print(top_predictions)

In [ ]:
# Save trained model to drive
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')

drive_path = '/content/gdrive/My Drive/saved_model/'

file_path = 'model_bert_full_more_more_data.pth'

# Save the model
torch.save(model.state_dict(), drive_path + file_path)